![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [201]:
import pandas as pd
import numpy as np

df = pd.read_csv('bank_marketing.csv')
df.insert(loc=11, column='year', value="2022")
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,year,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,2022,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,2022,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,2022,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,2022,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,2022,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [202]:
df.dtypes

client_id           int64
age                 int64
job                object
marital            object
education          object
credit_default     object
housing            object
loan               object
contact            object
month              object
day                 int64
year               object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                  object
dtype: object

In [203]:
rows, cols = df.shape
rows, cols

(41188, 23)

# Client dataframe

In [204]:
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client_df = client_df.rename(columns={'client_id': 'id'})
client_df.dtypes

id                 int64
age                int64
job               object
marital           object
education         object
credit_default    object
housing           object
loan              object
dtype: object

## Education analysis

In [205]:
client_df.education.value_counts()

university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

In [206]:
char_replace = lambda word: word.replace('.','_')
client_df['education'] = client_df.education.apply(char_replace)

In [207]:
replace_unknown = lambda row: np.nan if row == "unknown" else row
client_df['education'] = client_df.education.apply(replace_unknown)

In [208]:
client_df.education.value_counts()

university_degree      12168
high_school             9515
basic_9y                6045
professional_course     5243
basic_4y                4176
basic_6y                2292
illiterate                18
Name: education, dtype: int64

## Job analysis

In [209]:
drop_period = lambda word: word.replace(".", "")
client_df['job'] = client_df.job.apply(drop_period)

In [210]:
client_df.job.value_counts()

admin            10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [211]:
client_df.loan.value_counts()

no         33950
yes         6248
unknown      990
Name: loan, dtype: int64

## Columns to binarize

In [212]:
binarize = lambda row: 1 if row == 'yes' else 0
binarize('unknow')

0

In [213]:
binarize = lambda row: 1 if row == 'yes' else 0
client_df['credit_default'] = client_df.credit_default.apply(binarize)
client_df['housing'] = client_df.housing.apply(binarize)
client_df['loan'] = client_df.loan.apply(binarize)

In [214]:
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,0,0,0
1,1,57,services,married,high_school,0,0,0
2,2,37,services,married,high_school,0,1,0
3,3,40,admin,married,basic_6y,0,0,0
4,4,56,services,married,high_school,0,0,1


In [215]:
client_df.dtypes

id                 int64
age                int64
job               object
marital           object
education         object
credit_default     int64
housing            int64
loan               int64
dtype: object

# Campaingn dataframe

In [216]:
date_col = pd.to_datetime(df.year + "-" + df.month + "-" +df.day.astype("str"))
date_col.head()

0   2022-05-13
1   2022-05-19
2   2022-05-23
3   2022-05-27
4   2022-05-03
dtype: datetime64[ns]

In [217]:
campaign_df = df[['client_id',  'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y']]
campaign_df = campaign_df.rename(columns={"duration" : "contact_duration",
                                "previous" : "previous_campaign_contacts",
                                "y" : "campaign_outcome",
                                "poutcome" : "previous_outcome",
                                "campaign" : "number_contacts"})
campaign_df.insert(loc=7, column="last_contact_date", value=date_col)
campaign_df.insert(loc=0, column='campaign_id', value= np.ones((rows, 1)))
campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1.0,0,1,261,999,0,nonexistent,no,2022-05-13
1,1.0,1,1,149,999,0,nonexistent,no,2022-05-19
2,1.0,2,1,226,999,0,nonexistent,no,2022-05-23
3,1.0,3,1,151,999,0,nonexistent,no,2022-05-27
4,1.0,4,1,307,999,0,nonexistent,no,2022-05-03


## Previous outcome analysis

In [218]:
campaign_df.previous_outcome.value_counts()

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

In [219]:
drop_nonexistent = lambda row: np.nan if row == 'nonexistent' else row
campaign_df['previous_outcome'] = campaign_df.previous_outcome.apply(drop_nonexistent)

In [220]:
campaign_df.previous_outcome.value_counts()

failure    4252
success    1373
Name: previous_outcome, dtype: int64

In [221]:
binarize = lambda row: 1 if row == 'success' else 0
campaign_df['previous_outcome'] = campaign_df.previous_outcome.apply(binarize)

In [222]:
campaign_df.previous_outcome.value_counts()

0    39815
1     1373
Name: previous_outcome, dtype: int64

## Campaign outcome analysis

In [223]:
campaign_df.campaign_outcome.value_counts()

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [224]:
binarize = lambda row: 1 if row == 'yes' else 0
campaign_df['campaign_outcome'] = campaign_df.campaign_outcome.apply(binarize)

In [225]:
campaign_df.campaign_outcome.value_counts()

0    36548
1     4640
Name: campaign_outcome, dtype: int64

In [226]:
campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1.0,0,1,261,999,0,0,0,2022-05-13
1,1.0,1,1,149,999,0,0,0,2022-05-19
2,1.0,2,1,226,999,0,0,0,2022-05-23
3,1.0,3,1,151,999,0,0,0,2022-05-27
4,1.0,4,1,307,999,0,0,0,2022-05-03


In [227]:
campaign_df.dtypes

campaign_id                          float64
client_id                              int64
number_contacts                        int64
contact_duration                       int64
pdays                                  int64
previous_campaign_contacts             int64
previous_outcome                       int64
campaign_outcome                       int64
last_contact_date             datetime64[ns]
dtype: object

# Economics dataframe

In [228]:
economics_df = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]
economics_df = economics_df.rename(columns={"euribor3m": "euribor_three_months", 
                      "nr_employed": "number_employed"})
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [229]:
economics_df.dtypes

client_id                 int64
emp_var_rate            float64
cons_price_idx          float64
euribor_three_months    float64
number_employed         float64
dtype: object

# Save dataframes

In [230]:
client_df.to_csv('client.csv', index=False)
campaign_df.to_csv('campaign.csv', index=False)
economics_df.to_csv('economics.csv', index=False)

# Create database

In [231]:
client_table = """
CREATE TABLE client(
id SERIAL PRIMARY KEY,
age INTEGER,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOLEAN,
housing BOOLEAN,
loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [232]:
campaign_table = """
CREATE TABLE campaign(
campaign_id SERIAL PRIMARY KEY,
client_id SERIAL REFERENCES client (id),
number_contacts INTEGER,
contact_duration INTEGER,
pdays INTEGER,
previous_campaign_contacts INTEGER,
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE,
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [233]:
economics_table = """
CREATE TABLE economics(
client_id SERIAL REFERENCES client (id),
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT,
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""